In [15]:
from typing import Annotated, Sequence, TypedDict, List, Dict
from dotenv import load_dotenv  
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage, SystemMessage
from langchain_groq import ChatGroq
from langchain_core.tools import tool
from langgraph.graph.message import add_messages
from langgraph.graph import StateGraph, END, START
from langgraph.prebuilt import ToolNode
import os

In [16]:
load_dotenv()
GROQ_API_KEY=os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"]= GROQ_API_KEY

In [17]:
llm = ChatGroq(model="llama-3.1-8b-instant")

In [18]:
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    subtopic: List[str]
    slide_segments: List[Dict[str, str]]
    ppt_output_path: str

In [19]:
import json
from typing import List, Dict, TypedDict, Sequence, Annotated

def load_json_to_agent_state(json_path: str) -> AgentState:
    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    # Ensure all slide segments have the necessary fields
    complete_slide_segments = []
    for slide in data.get('slide_segments', []):
        complete_slide = {
            "slide_no": slide.get("slide_no", 0),
            "subtopic": slide.get("subtopic", ""),
            "content_to_display": slide.get("content_to_display", ""),
            "narration_script": slide.get("narration_script", ""),
            "is_blank_slide": slide.get("is_blank_slide", False),
            "image_address": slide.get("image_address", ""),
            "video_address": slide.get("video_address", ""),
            "image_position": slide.get("image_position", ""),
            "content_position": slide.get("test_position", "")  # mapping test_position to content_position
        }
        complete_slide_segments.append(complete_slide)

    agent_state: AgentState = {
        "messages": [],
        "subtopic": data.get('subtopics', []),
        "slide_segments": complete_slide_segments
    }

    return agent_state


# Example usage
json_path = '../assets/scripts/slide_segments.json'
agent_state = load_json_to_agent_state(json_path)
print(agent_state)


{'messages': [], 'subtopic': ['1. Rate of Reaction', '2. Reaction Mechanisms', '3. Catalysts and Inhibitors', '4. Temperature Effects', '5. Activation Energy', '6. Order of Reaction', '7. Molecularity'], 'slide_segments': [{'slide_no': 1, 'subtopic': '1. Rate of Reaction', 'content_to_display': '1. The rate of reaction refers to the speed at which reactants are converted into products.\n2. The rate of reaction can be affected by various factors including the nature of the reaction, the physical state of reactants, concentration, pressure, temperature, and the presence of a solvent or electromagnetic radiation.\n3. The rate of reaction can be increased by increasing the concentration of reactants, applying pressure, increasing temperature, and exposing the reactants to electromagnetic radiation.', 'narration_script': 'The rate of reaction is the speed at which reactants are converted into products. For example, when you burn wood, the rate of reaction is very fast because the wood is co

In [20]:
import requests

PEXELS_API_KEY = 'mIZdthiPsT6hrIGHcGTOgwH61Q4UvepeUP3o9sU9GUqXm1HVhqas1fQQ'
PEXELS_API_URL = 'https://api.pexels.com/v1/search'


def get_relevant_image(query: str, per_page: int = 1) -> str:
    headers = {
        'Authorization': PEXELS_API_KEY
    }
    params = {
        'query': query,
        'per_page': per_page
    }

    response = requests.get(PEXELS_API_URL, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['photos']:
            # Get the first photo's large image URL
            return data['photos'][0]['src']['large']
        else:
            print("No images found for query:", query)
            return ""
    else:
        print(f"Error {response.status_code}: {response.text}")
        return ""


import requests

PEXELS_API_KEY = 'mIZdthiPsT6hrIGHcGTOgwH61Q4UvepeUP3o9sU9GUqXm1HVhqas1fQQ'
PEXELS_VIDEO_API_URL = 'https://api.pexels.com/videos/search'


def get_relevant_video(query: str, per_page: int = 1) -> str:
    headers = {
        'Authorization': PEXELS_API_KEY
    }
    params = {
        'query': query,
        'per_page': per_page
    }

    response = requests.get(PEXELS_VIDEO_API_URL, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        if data['videos']:
            # Get the first video's link (e.g., highest quality or medium)
            return data['videos'][0]['video_files'][0]['link']
        else:
            print("No videos found for query:", query)
            return ""
    else:
        print(f"Error {response.status_code}: {response.text}")
        return ""


In [21]:
def enrich_all_slides_with_media(agent_state: dict) -> dict:
    """
    Enriches all slide segments in the agent_state with images or videos
    from Pexels based on whether the slide is blank or has content.
    """

    enriched_segments = []

    for slide in agent_state.get('slide_segments', []):
        content = slide.get('content_to_display', "")
        is_blank = slide.get('is_blank_slide', False)

        if is_blank:
            video_url = get_relevant_video(content)
            slide['video_address'] = video_url
            slide['image_address'] = ""
            slide['image_position'] = ""
            slide['content_position'] = "center"
        else:
            image_url = get_relevant_image(content)
            slide['image_address'] = image_url
            slide['video_address'] = ""
            slide['image_position'] = "right"
            slide['content_position'] = "left"

        enriched_segments.append(slide)

    # Update agent_state
    agent_state['slide_segments'] = enriched_segments
    return agent_state


In [22]:
enriched_state = enrich_all_slides_with_media(agent_state)
print(enriched_state)


No videos found for query: BLANK_SLIDE
{'messages': [], 'subtopic': ['1. Rate of Reaction', '2. Reaction Mechanisms', '3. Catalysts and Inhibitors', '4. Temperature Effects', '5. Activation Energy', '6. Order of Reaction', '7. Molecularity'], 'slide_segments': [{'slide_no': 1, 'subtopic': '1. Rate of Reaction', 'content_to_display': '1. The rate of reaction refers to the speed at which reactants are converted into products.\n2. The rate of reaction can be affected by various factors including the nature of the reaction, the physical state of reactants, concentration, pressure, temperature, and the presence of a solvent or electromagnetic radiation.\n3. The rate of reaction can be increased by increasing the concentration of reactants, applying pressure, increasing temperature, and exposing the reactants to electromagnetic radiation.', 'narration_script': 'The rate of reaction is the speed at which reactants are converted into products. For example, when you burn wood, the rate of react

In [11]:
!pip install python-pptx


  Obtaining dependency information for python-pptx from https://files.pythonhosted.org/packages/d9/4f/00be2196329ebbff56ce564aa94efb0fbc828d00de250b1980de1a34ab49/python_pptx-1.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for XlsxWriter>=0.5.7 from https://files.pythonhosted.org/packages/fa/34/a22e6664211f0c8879521328000bdcae9bf6dbafa94a923e531f6d5b3f73/xlsxwriter-3.2.5-py3-none-any.whl.metadata
Using cached python_pptx-1.0.2-py3-none-any.whl (472 kB)
Using cached xlsxwriter-3.2.5-py3-none-any.whl (172 kB)


In [23]:
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.enum.text import MSO_ANCHOR
import requests
from io import BytesIO
import os



def create_ppt_with_alignment(agent_state: dict, output_path: str = None) -> dict:
    """
    Generates a PPT using python-pptx from agent_state's slide_segments.
    Aligns image and text based on 'image_position'.
    """

    if not output_path:
        output_path = os.path.join(os.getcwd(), 'GeneratedPresentation.pptx')

    prs = Presentation()

    for slide_data in agent_state.get('slide_segments', []):
        slide = prs.slides.add_slide(prs.slide_layouts[5])  # Blank layout for custom placement

        # Title
        title_shape = slide.shapes.add_textbox(Inches(0.5), Inches(0.3), Inches(9), Inches(1))
        title_frame = title_shape.text_frame
        title_frame.text = slide_data.get('subtopic', '')
        title_frame.paragraphs[0].font.size = Pt(24)

        # Content
        content_text = slide_data.get('content_to_display', '')
        image_url = slide_data.get('image_address', '')
        image_position = slide_data.get('image_position', 'right')

        # Define sizes
        image_width = Inches(3)
        text_width = Inches(5.5)
        height = Inches(4)

        if image_position == 'left':
            # Image on left, text on right
            img_left = Inches(0.5)
            text_left = img_left + image_width + Inches(0.5)
        else:
            # Image on right, text on left
            text_left = Inches(0.5)
            img_left = text_left + text_width + Inches(0.5)

        # Add text
        text_box = slide.shapes.add_textbox(text_left, Inches(1.5), text_width, height)
        text_frame = text_box.text_frame
        text_frame.text = content_text
        text_frame.word_wrap = True
        text_frame.vertical_anchor = MSO_ANCHOR.TOP
        text_frame.paragraphs[0].font.size = Pt(18)

        # Add Image
        if image_url:
            try:
                response = requests.get(image_url)
                if response.status_code == 200:
                    image_stream = BytesIO(response.content)
                    slide.shapes.add_picture(image_stream, img_left, Inches(1.5), width=image_width)
            except Exception as e:
                print(f"Failed to add image for slide {slide_data.get('slide_no')}: {e}")

    prs.save(output_path)
    print(f"PPT created with alignment at: {output_path}")

    agent_state['ppt_output_path'] = output_path
    return agent_state


In [24]:
output_state = create_ppt_with_alignment(agent_state, output_path='../assets/ppts/GeneratedPresentation.pptx')
print("Generated PPT at:", output_state['ppt_output_path'])


PPT created with alignment at: ../assets/ppts/GeneratedPresentation.pptx
Generated PPT at: ../assets/ppts/GeneratedPresentation.pptx
